# Semester Project Notebook - EPFL HP

In [27]:
# Libraries
import pandas as pd
import numpy as np

# 01 - Fluid selection

### Legislation: banned fluids (Montreal Protocol, OFEV)
Fluid with ozone depletion potention, global warming potential and stable in the atmosphere.
- HCFC 
- HCFO
- CFC 
- HFC 

### Environmental Criteria
- GWPmax = 400 ?
- ODPmax = 0

### Operating Criteria
- Tcrit,min  = 90°C (to avoid entering the supercritical domain)
- Teb,max = 10°C (if Teb too high, very low pressure required for evaporation)


### Fluid screening
Selected for the study: Fluids figuring in the (non-exhaustive) OFEV's list of autorized fluids:
- R-290 (propane, HC)
- R-600 (butane, HC)
- R-600a (isobutane, HC)
- R-1270 (propylène, HO)
- R-1234yf (HFO)
- R-1234ze (HFO)
- R-717 (ammonia, N)

Not selected for the study: Fluids not in the list:
- R-630 (methylamine, HC)
- R-40 (HCC)
- R-1140 (HCO)
- R-764 (sulfure dioxide, N)

### Physical properties for unknown fluids in Belsim VALI (R-1234yf, R-1234ze)
- Thermodynamic models: VIDEAL/LIDEAL or Soave/Peng-Robinson
- Physical properties required: critical temperature, critical pressure, boiling curve coefficients, ascentric factor

In [16]:
properties = {'ASHRAE': ['R-1234yf','R-1234ze'], 'Formula':['C3F4H2','C3F4H2'], 'Tcrit [K]':[367.85,423.27], 'Pcrit [bar]':[33.822, 35.349], 'PVAP':['?','?'], 'omega':[0.276,0.326]}
properties = pd.DataFrame(properties)
properties = properties.set_index('ASHRAE')
properties

,Formula,Tcrit [K],Pcrit [bar],PVAP,omega
ASHRAE,,,,,
R-1234yf,C3F4H2,367.85,33.822,?,0.276
R-1234ze,C3F4H2,423.27,35.349,?,0.326


# 02 - Heating demand

Monthly mean heating demand during operating hours for the two heating loops (lowT (50°C) and mediumT (65°C)) of EPFL's heating system. 

In [35]:
heating_demand = pd.read_csv('Data/heating_monthly.csv', sep=';')
heating_demand = heating_demand.set_index('Month')
heating_demand['MT'] = heating_demand['MT'].apply(lambda x: x/1000 if x > 0 else 0)
heating_demand['LT'] = heating_demand['LT'].apply(lambda x: x/1000 if x > 0 else 0)
heating_demand.rename(columns={'MT':'MT [kW]', 'LT':'LT [kW]'})

,MT [kW],LT [kW]
Month,,
1,7395.670705,4318.682087
2,5534.986003,3283.383266
3,2904.392606,1811.057933
4,43.868802,192.982501
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,0.000000
9,0.000000,36.520003


# 03 - Optimal condensation temperature (HP_T_6)

### Working assumptions
- HP design optimized for colder working conditions (Q_ cond = 6 MW)
- isentropic compressor efficiency: 0.85
- same pressure ratio for the 2 compressors
- boiling temperature at the evaporator is constant (6°C)
- water enters in EPFL with T_EPFL_in = 65/50 °C and leaves EPFL with T_EPFL_out = 30°C
- CO2 emissions from gird electricity: 0.206 kgCO2eq/kWh
- swiss CO2 tax: 100 CHF/t CO2
- electricity price: 0.0916 CHF/kWh
- lifetime = 20 years

### Flowchart of the Belsim VALI Model
Imposed conditions (CST) are marked in yellow

<img src=flowchart_optimal_t_hp_6.png>

### Compressor and heat exchanger cost calculation (CAPEX)

$A_{HEX} = \frac{Q_{cond}}{U \cdot LMDT} $

$C_p = 10^{K_1 + K_2\cdot \log(A) + K_3 \cdot \log(A)^2} \cdot \frac{I_t}{I_{t,ref}}$
where $A$ is the HEX area in [m2] or the compressor power in [kW].

$C = F_{BM} \cdot C_p$

$CAPEX = C \cdot \frac{i(i+1)^n}{(i+1)^n-1}$ [CHF/year]

In [70]:
cost_parameters = {'Equipment': ['Compressor','Condensor','Evaporator'], 'Fbm':[2.45,4.74,4.74], 'K1':[2.2897,3.2138,3.5638], 'K2':[1.3604,0.2688,0.1906], 'K3':[-0.1027,0.07961,0.1107], 'Iref':[394.3,394.3,394.3], 'It':[584.6, 584.6,584.6]}
cost_parameters = pd.DataFrame(cost_parameters)
cost_parameters = cost_parameters.set_index('Equipment')
cost_parameters

,Fbm,K1,K2,K3,Iref,It
Equipment,,,,,,
Compressor,2.45,2.2897,1.3604,-0.10270,394.3,584.6
Condensor,4.74,3.2138,0.2688,0.07961,394.3,584.6
Evaporator,4.74,3.5638,0.1906,0.11070,394.3,584.6


### Operation costs (OPEX)

$OPEX = \sum_i^{period} E_i (c_{el} + grid_{CO_2} \cdot tax_{CO2})$

### Results from VALI
Iteration of the model over T_HP_6 for each typical period

In [67]:
result_VALI = pd.read_csv('Data/R-717_all_Results.csv', sep=',')
result_VALI

,month,W_COMP_1,W_COMP_2,Q_COND_LOAD,Q_EVAP_LOAD,T_HP_5,T_HP_6
0,1,289.092,326.610,6000.0,5384.30,49.7586,30
1,1,299.751,340.400,6000.0,5359.85,51.5175,31
2,1,310.304,354.207,6000.0,5335.49,53.2714,32
3,1,320.753,368.031,6000.0,5311.22,55.0201,33
4,1,331.098,381.872,6000.0,5287.03,56.7639,34
...,...,...,...,...,...,...,...
107,12,400.651,479.268,6000.0,5120.08,68.8302,41
108,12,410.185,493.260,6000.0,5096.56,70.5342,42
109,12,419.620,507.273,6000.0,5073.11,72.2333,43
110,12,428.957,521.307,6000.0,5049.74,73.9276,44


### Computations and plot for optimization

In [71]:
# Condenser design for maximal load
T_EPFL_MT_in = 65
T_EPFL_LT_in = 50
T_EPFL_out = 30
U = 0.75
data_max_load = result_VALI[result_VALI['month']==1]
data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
# Filtering out not possible T conditions
data_max_load = data_max_load.dropna()
data_max_load['A_condensor'] = data_max_load.apply(lambda x: x.Q_COND_LOAD/U/x.LMTD, axis=1)

# Evaporator design
LMTD_evap = 

# Cost condensor
data_max_load['c_HEX_cond'] = data_max_load.apply(lambda x: cost_parameters.loc['Condensor','It']/cost_parameters.loc['Condensor','Iref']*cost_parameters.loc['Condensor','Fbm']*(10**(cost_parameters.loc['Condensor','K1']+cost_parameters.loc['Condensor','K2']*np.log10(x.A_condensor)+cost_parameters.loc['Condensor','K3']*(np.log10(x.A_condensor)**2))),axis=1)
#c_HEX_evap = 
data_max_load

<ipython-input-71-a104f6b30d2e>:7: RuntimeWarning: invalid value encountered in log
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
<ipython-input-71-a104f6b30d2e>:7: RuntimeWarning: divide by zero encountered in log
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
<ipython-input-71-a104f6b30d2e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),

,month,W_COMP_1,W_COMP_2,Q_COND_LOAD,Q_EVAP_LOAD,T_HP_5,T_HP_6,LMTD,A_condensor,c_HEX_cond
9,1,381.284,451.345,6000.0,5167.37,65.4074,39,2.776119,2881.720878,877873.893391
10,1,391.017,465.296,6000.0,5143.69,67.1213,40,5.081210,1574.428253,541674.207233
11,1,400.651,479.268,6000.0,5120.08,68.8302,41,6.796159,1177.135487,433280.200705
12,1,410.185,493.260,6000.0,5096.56,70.5342,42,8.354183,957.604168,371094.636209
13,1,419.620,507.273,6000.0,5073.11,72.2333,43,9.836521,813.295637,328961.985223
14,1,428.957,521.307,6000.0,5049.74,73.9276,44,11.274262,709.580834,297883.613551
15,1,438.197,535.364,6000.0,5026.44,75.6170,45,12.682522,630.789343,273722.267693


In [36]:
cost_parameters.loc['Condensor','Fbm']

4.74

In [41]:
cost_parameters.loc['Condensor','K1']*np.log(2)


2.2276364088835523

In [57]:
np.log(data_max_load.loc[9,'A_condensor'])**2

63.459433052499165

In [48]:
T_EPFL_MT_in = 65
T_EPFL_LT_in = 50
T_EPFL_out = 30
U = 0.75
data_max_load = result_VALI[result_VALI['month']==1]
data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
#data_max_load = data_max_load[data_max_load['LMTD'] != 'NaN']

data_max_load.dropna()

<ipython-input-48-6acd684fc24a>:6: RuntimeWarning: invalid value encountered in log
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
<ipython-input-48-6acd684fc24a>:6: RuntimeWarning: divide by zero encountered in log
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),axis=1)
<ipython-input-48-6acd684fc24a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_max_load['LMTD'] = data_max_load.apply(lambda x: ((x.T_HP_5-T_EPFL_MT_in)-(x.T_HP_6-T_EPFL_out))/(np.log(x.T_HP_5-T_EPFL_MT_in)-np.log(x.T_HP_6-T_EPFL_out)),

,month,W_COMP_1,W_COMP_2,Q_COND_LOAD,Q_EVAP_LOAD,T_HP_5,T_HP_6,LMTD
9,1,381.284,451.345,6000.0,5167.37,65.4074,39,2.776119
10,1,391.017,465.296,6000.0,5143.69,67.1213,40,5.081210
11,1,400.651,479.268,6000.0,5120.08,68.8302,41,6.796159
12,1,410.185,493.260,6000.0,5096.56,70.5342,42,8.354183
13,1,419.620,507.273,6000.0,5073.11,72.2333,43,9.836521
14,1,428.957,521.307,6000.0,5049.74,73.9276,44,11.274262
15,1,438.197,535.364,6000.0,5026.44,75.6170,45,12.682522
